In [1]:
############################################
# Section 0

# Matplotlib Inline
%matplotlib inline

# Import libraries
import os
import shutil
import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import re # regular expressions
import collections
import pandas as pd
from scipy import stats
import subprocess
import itertools
import time

# Import function file
from ic_functions_final import *

# Change pd settings to display all dataframe content
pd.set_option('display.max_rows',None)

# Close all pre-existing figures
plt.close('all')

In [2]:
############################################
## Section 0: Gene Identifiers Dictionary
geneID,NametoIDs,NametoFBid = geneIDdictionary('Datasets/fbgn_annotation_ID_fb_2016_05.tsv')

## Section 1: Look at Information Content of TFs

# Read PWM data from FlyFactor Survey
# PWM,checkFBgn = PWMdictionary("Datasets/UmassPGFE_PWMfreq_PublicDatasetA_20150901.txt")
# PWM,checkFBgn = readFASTA("Datasets/UmassPGFE_PWMfreq_PublicDatasetA_20150901.txt", \
#                           re.compile('FBgn\d*'),'PWM','v',geneID)
PWM,checkFBgn,PWMnames = readFASTA('Datasets/UmassPGFE_PWMfreq_PublicDatasetA_20150901.txt', \
                          'PWM','v',geneID,re.compile('(FBgn\d*)'),pseudocount=0)
# Discrepancy Checks:
# checkFBgn = 'FBgn0050420', 'FBgn0051782'
# 'FBgn0050420': FBgn0050420 associated with CG44246(Atf-2) and CG44247;
#                confirmed that protein fragment characterized is in Atf-2: 283-336
#                Note that UniProt identifies 281 – 339 as the BZIP
# 'FBgn0051782': obsolete protein according to UniProt
#                currently references three adjacent genes
#                THIS SHOULD BE REMOVED.
PWM.pop(geneID['FBgn0051782'][0],None)
PWMnames.pop([k for k,v in PWMnames.iteritems() if 'FBgn0051782' in v][0])
# Replace da PWM with the da-specific PWM
da,checkFBgn_da,PWMnames_da = readFASTA('Datasets/UmassPGFE_PWMfreq_da_SANGER_10_Vertical20160731.txt', \
                          'PWM','v',geneID,re.compile('(FBgn\d*)'))
PWM.update(da)
PWMnames.update(PWMnames_da)
gt,checkFBgn_gt,PWMnames_gt = readFASTA('Datasets/UmassPGFE_PWMfreq_gt_FlyReg_Vertical20160810.txt', \
                          'PWM','v',geneID,re.compile('(FBgn\d*)'))
PWM.update(gt)
PWMnames.update(PWMnames_gt)


# First, remove FBgn from the end of the keys because that's how the aligned sequences are
# labeled in the file of aligned sequences: UmassPGFE_PWMfreq_PublicDatasetD_20160225.txt
filteredPWMnames = {}  # initialize dictionary
filteredPWMnamesR = {}
regex = re.compile('([\w\.\(\)-]+)_')
for k,v in PWMnames.iteritems():
    m = regex.search(v)
    filteredPWMnames[k] = m.group(1)
    if m.group(1) in filteredPWMnamesR:
        filteredPWMnamesR[m.group(1)].append(k)
    else:
        filteredPWMnamesR[m.group(1)] = [k]

In [3]:
# ONLY NEEDS TO BE RUN ONCE

# Section 2: Determine and write aligned FlyReg sequences to the aligned sequence files
# Read in the footprints for the TFs that only have PWMs based on FlyReg footprints: brk, Mad, Med
regex = re.compile('(\w+)\tBergman_data\tbinding_site\t(\d+)\t(\d+)\t\.\t\.\t\.\tFactor\s([\w\"\-\(\)\.]+)')
# group(1) = chromosome
# group(2) = start
# group(3) = end
# group(4) = TF

if not os.path.exists('Datasets/Modified'):
    os.makedirs('Datasets/Modified')

FlyRegTFs = []
with open('Datasets/Modified/FlyRegTFs.bed','w') as f:
    with open('Datasets/Footprint.GFF','r') as f1:
        for line in f1:
            m = regex.search(line)
            if m:
                if m.group(4)[1:-1] not in FlyRegTFs:
                    FlyRegTFs.append(m.group(4)[1:-1])
                f.write('chr%s\t%s\t%s\t%s\n' %(m.group(1),m.group(2),m.group(3),m.group(4)[1:-1]))
FlyRegTFs = sorted(FlyRegTFs, key=lambda t:t.lower())

# Liftover from dm2 (Release 4) to dm6
command = []
# Liftover from dm2 to dm3
command.append(['./liftOver', \
                'Datasets/Modified/FlyRegTFs.bed', \
                'Datasets/dm2ToDm3.over.chain.gz', \
                'Datasets/Modified/FlyRegTFs_dm3.bed', \
                'Datasets/Modified/FlyRegTFs_unlifted.bed'])
# Liftover from dm3 to dm6
command.append(['./liftOver', \
                'Datasets/Modified/FlyRegTFs_dm3.bed', \
                'Datasets/dm3ToDm6.over.chain.gz', \
                'Datasets/Modified/FlyRegTFs_dm6.bed', \
                'Datasets/Modified/FlyRegTFs_unlifted.bed'])
for cmd in command:
    process = subprocess.call(cmd)
    # Sanity check: no coordinates unlifted
    if os.stat('Datasets/Modified/FlyRegTFs_unlifted.bed').st_size == 0:
        print 'No coordinates unlifted.'
    else:
        print 'Some coordinates unlifted!'

# Pick out the sequences from dm6 and run patser on them
# Remove 'Unspecified' because those sequences don't pertain to the PWM of any TF
if 'Unspecified' in FlyRegTFs:
    FlyRegTFs.remove('Unspecified')
# For names from the Fly Reg footprint database that don't match with those from the FlyReg PWM database, 
# provide translation dictionary
translations = dict(zip(['abd-A','E(spl)','Su(H)','su(Hw)'],['Abd-A','Espl','SuH','suHw']))
temp = list(set(FlyRegTFs)-set(translations.keys()))
translations.update(dict(zip(temp,temp)))
# For names from the Fly Reg footprint database with the gene symbol designated in Flybase/PWM database, 
# provide translation dictionary
keys = sorted([TF for TF in FlyRegTFs if TF not in PWM.keys()], key=lambda t: t.lower())
values = ['BEAF-32','BEAF-32','BEAF-32','br','br','br','br','Cf2','dsx','dsx','dwg','E(spl)m8-HLH','His2B','E(spl)m5-HLH','Myb','p120','Top2','Trf','zfh1']
translations1 = dict(zip(keys,values))
temp = list(set(FlyRegTFs)-set(keys))
translations1.update(dict(zip(temp,temp)))
# Save the raw sequences for later use
Raw_Seq = {}
# Pull out the FlyReg PWMs (pseudocount = 0)
PWM_FlyReg,checkFBgn_FlyReg,PWMnames_FlyReg = readFASTA('Datasets/UmassPGFE_PWMfreq_PublicDatasetA_20150901.txt', \
                                                'PWM','v',geneID,re.compile('([\w\-\.\(\)]+)_FlyReg_FBgn\d*'),nameasis=1,pseudocount=0)
# Write the FlyReg PWMs to file to be used with patser
### make the matrix files for PATSER
bases = ['A','C','G','T']
for key in PWM_FlyReg:
    if not os.path.exists('matrix/FlyReg'):
        os.makedirs('matrix/FlyReg')
    with open('matrix/FlyReg/%s.txt' %key, 'w') as f:       
        with open('matrix/FlyReg/%s.txt' %key, 'a') as f:
            for i in range(0,4): #for each base
                temp = ['%s |' %bases[i]] + \
                       map(str,np.transpose(PWM_FlyReg[key])[i,:].tolist())
                temp1 = ''.join([x+'\t' for x in temp]) + '\n'
                f.write(temp1)

# Read in the dm6 genome
dm6 = readFASTA('Datasets/dm6.fa',0,'sequences',0,0,0)
# Create directories for raw sequences and patser-processed raw sequences
directories = ['patserIn/RawSeq', \
               'patserOut/RawSeq']
for d in directories:
    if not os.path.exists(d):
        os.makedirs(d)
# path to patser
path_to_patser = 'patser-v3e.1/patser-v3e'
# 1) Read in the raw sequences, 2) write to TF-specific raw sequence files, and 3) run patser
for TF in FlyRegTFs:
    ctr = 0 # counter for the sequence
    with open('patserIn/RawSeq/RawSeq_%s.txt' %TF,'w') as f1:
        with open('Datasets/Modified/FlyRegTFs_dm6.bed','r') as f:
            for line in f:
                temp = line.strip().split('\t')
                # temp[0] = chromosome
                # temp[1] = start
                # temp[2] = end
                # temp[3] = TF name
                if temp[3] == TF:
                    ctr +=1
                    try:
                        # if the raw sequence is shorter than the PWM, add +/- 10bp to the raw sequence
                        if int(temp[2])-int(temp[1])+1 < PWM_FlyReg[translations[TF]].shape[0]:
                            f1.write('%s_%s \ %s \ \n' %(temp[3],ctr, dm6[temp[0]][int(temp[1])-11:int(temp[2])+10]))
                            Raw_Seq['%s_%s' %(temp[3],ctr)] = dm6[temp[0]][int(temp[1])-11:int(temp[2])+10]
                        else: # write raw sequence as is
                            f1.write('%s_%s \ %s \ \n' %(temp[3],ctr, dm6[temp[0]][int(temp[1])-1:int(temp[2])]))
                            Raw_Seq['%s_%s' %(temp[3],ctr)] = dm6[temp[0]][int(temp[1])-1:int(temp[2])]
                    except KeyError: # if no PWM exists for these sequences, write raw sequences to file as is
                        print '[KeyError]: %s is not in PWM_FlyReg' %TF
                        f1.write('%s_%s \ %s \ \n' %(temp[3],ctr, dm6[temp[0]][int(temp[1])-1:int(temp[2])]))
                        Raw_Seq['%s_%s' %(temp[3],ctr)] = dm6[temp[0]][int(temp[1])-1:int(temp[2])]

    # run Patser using FlyReg PWMs
    if translations[TF] in PWM_FlyReg.keys():
        matrix = 'matrix/FlyReg/%s.txt' %translations[TF] # path to FlyReg TF PWM file
        inputFile = 'patserIn/RawSeq/RawSeq_%s.txt' %TF # input sequences
        outputFile = 'patserOut/RawSeq/RawSeq_%s.txt' %TF # PATSER output

        cmd = [path_to_patser, '-A', 'a:t', '0.297', 'c:g', '0.203', \
               '-p', '-m', matrix, '-b', '0.01', '-c', '-d1', '-t', '-f', inputFile] 

        process = subprocess.call(cmd,stdout = open(outputFile, 'w'))
        print('Done: %s' %TF)

# First pass at using patser results to determine aligned sequences
AlignedSeq_FlyReg = {} # dictionary of aligned sequences (in case there are repeats)
mismatch_AlignedSeq = [] # list of PWMs for which some raw sequences have not been used, i.e. n_raw seq > n_aligned seq
missing_AlignedSeq = [] # list of PWMs for which some aligned sequences are missing
temp_PWM_FlyReg = {} # dictionary of PWMs created from aligned sequences
RC = {'A':'T','T':'A','C':'G','G':'C'} # reverse complement dictionary
index = {'A':0,'C':1,'G':2,'T':3}
regex = re.compile('([\w\-\.\(\)]+)\s+position=\s+([\dC]+)')
# group(1) = enhancer, or sequence name
# group(2) = position
for TF in FlyRegTFs:
    PWM_TF = translations[TF]
    if PWM_TF in PWM_FlyReg.keys(): # if a FlyReg PWM exists for this TF
        AlignedSeq_FlyReg[TF] = {} # initialize FlyReg aligned sequence dictionary for TF
        ctr = 0 # counter for saved sequences
        temp_PWM = np.zeros((PWM_FlyReg[PWM_TF].shape)) # initialize PWM created with these aligned sequences
        with open('patserOut/RawSeq/RawSeq_%s.txt' %TF,'r') as f:
            for line in f:
                m = regex.search(line)
                if m:
                    save = 0
                    if 'C' in m.group(2):
                        i = int(m.group(2)[:-1]) # position in the sequence
                        temp_seq = ''.join([RC[base.upper()] for base in Raw_Seq[m.group(1)]])[i-1:i-1+PWM_FlyReg[PWM_TF].shape[0]][::-1].upper()
                        if m.group(1) not in AlignedSeq_FlyReg[TF].keys():
                            ctr1,save = 1,1 # counter for raw sequence, use this sequence in PWM
                            AlignedSeq_FlyReg[TF].update({m.group(1):temp_seq})
                            ctr += 1 # increment counter of all saved sequences
                        elif (m.group(1) in AlignedSeq_FlyReg[TF].keys() and temp_seq not in [seq for tf,seq in AlignedSeq_FlyReg[TF].iteritems() if m.group(1) in tf]):
                            ctr1 += 1 # counter for raw sequence incremented if same sequence has more than one alignment
                            save = 1
                            AlignedSeq_FlyReg[TF].update({'%s_%d' %(m.group(1),ctr1):temp_seq})
                            ctr += 1
                    else:
                        i = int(m.group(2)) # position in the sequence
                        temp_seq = Raw_Seq[m.group(1)][i-1:i-1+PWM_FlyReg[PWM_TF].shape[0]].upper()
                        if m.group(1) not in AlignedSeq_FlyReg[TF].keys():
                            ctr1,save = 1,1
                            AlignedSeq_FlyReg[TF].update({m.group(1):temp_seq})
                            ctr += 1
                        elif (m.group(1) in AlignedSeq_FlyReg[TF].keys() and temp_seq not in [seq for tf,seq in AlignedSeq_FlyReg[TF].iteritems() if m.group(1) in tf]):#AlignedSeq_FlyReg[TF][m.group(1)] != temp_seq):
                            ctr1 += 1
                            save = 1
                            AlignedSeq_FlyReg[TF].update({'%s_%d' %(m.group(1),ctr1):temp_seq})
                            ctr += 1
                    if save == 1: # if aligned sequence is unique or raw seq produced multiple alignments
                        for position,base in enumerate(temp_seq):
                            temp_PWM[position,index[base]] += 1

        temp_PWM_FlyReg[PWM_TF] = temp_PWM
        if not np.array_equal(temp_PWM,PWM_FlyReg[PWM_TF]):
            mismatch_AlignedSeq.append(TF)
            print '%s: %d aligned sequences/%d in PWM' %(TF,ctr,math.floor(sum(PWM_FlyReg[PWM_TF][0]))) 
        if ctr < math.floor(sum(PWM_FlyReg[translations[TF]][0])):
            missing_AlignedSeq.append(TF)
            print '**%s is missing %d aligned sequences.**' %(TF,math.floor(sum(PWM_FlyReg[PWM_TF][0]))-ctr)  

# For those PWMs that don't use all the raw sequences, determine which combination produces the PWM
subsets = {}
# for TF in AlignedSeq_FlyReg.keys():
for TF in ['abd-A','ap','br-Z2','Dref','eve','grh','Kr','ttk']:
# for TF in ['abd-A']:
    timeout = time.time() + 60
    if len(AlignedSeq_FlyReg[TF]) != int(math.floor(sum(PWM_FlyReg[translations[TF]][0]))):
        print TF
        for subset in itertools.combinations(AlignedSeq_FlyReg[TF].keys(), int(math.floor(sum(PWM_FlyReg[translations[TF]][0])))):
            temp_PWM = np.zeros((PWM_FlyReg[translations[TF]].shape))
            for seq in subset:
                for position,base in enumerate(AlignedSeq_FlyReg[TF][seq]):
                    temp_PWM[position,index[base]] += 1
            if np.array_equal(temp_PWM,PWM_FlyReg[translations[TF]]):
                if TF in subsets.keys():
                    subsets[TF].append([subset])
                else:
                    subsets[TF] = [subset]
                temp_PWM_FlyReg[translations[TF]] = temp_PWM
                print sorted(subset)
            if time.time() > timeout:
                break
        if TF != 'br-Z2':
            AlignedSeq_FlyReg[TF] = {k:v for k,v in AlignedSeq_FlyReg[TF].iteritems() if k in subsets[TF][0]}
        else:
            AlignedSeq_FlyReg[TF] = {k:v for k,v in AlignedSeq_FlyReg[TF].iteritems() if k in subsets[TF][1]}

# Add the missing sequence for the Mad PWM
temp = PWM['Mad']-temp_PWM_FlyReg['Mad']
indexRev = {v:k for k,v in index.iteritems()}
AlignedSeq_FlyReg['Mad']['Mad_20'] = ''.join(indexRev[[idx for idx, val in enumerate(temp[i]) if val >= 1][0]] for i in range(len(PWM['Mad'])))
            
# Create patserIn/AlignedSeq if it does not exist
if not os.path.exists('patserIn/AlignedSeq'):
    os.makedirs('patserIn/AlignedSeq')

translationsRev = {v:k for k,v in translations1.iteritems()}
# filteredPWMnamesRev = {v:k for k,v in filteredPWMnames.iteritems()}
od_AlignedSeq_FlyReg = {TF:collections.OrderedDict([(k,seq[k]) for k in sorted(seq,key=lambda string: [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string)])]) for TF,seq in AlignedSeq_FlyReg.iteritems()}
# for the FlyReg PWMs in PWM dictionary minus those whose raw sequences I was unable to align
for TF in list(set([k for k,v in PWMnames.iteritems() if 'FlyReg' in v])-(set(mismatch_AlignedSeq)-set(['Mad'])-set(subsets.keys()))):
    ctr = 1
    FlyReg_TF = translationsRev[TF]
    with open('patserIn/AlignedSeq/%s.txt' %TF,'w') as f:
        for seq in od_AlignedSeq_FlyReg[FlyReg_TF].values():
            f.write('%s_%d \ %s \ \n' %(filteredPWMnames[TF],ctr,seq))
            ctr +=1

No coordinates unlifted.
No coordinates unlifted.
Done: abd-A
Done: Abd-B
Done: Adf1
Done: Aef1
Done: Antp
Done: ap
[KeyError]: ara is not in PWM_FlyReg
[KeyError]: ara is not in PWM_FlyReg
Done: bab1
[KeyError]: bap is not in PWM_FlyReg
Done: bcd
[KeyError]: BEAF-32 is not in PWM_FlyReg
[KeyError]: BEAF-32 is not in PWM_FlyReg
[KeyError]: BEAF-32A is not in PWM_FlyReg
[KeyError]: BEAF-32A is not in PWM_FlyReg
[KeyError]: BEAF-32B is not in PWM_FlyReg
[KeyError]: BEAF-32B is not in PWM_FlyReg
Done: bin
Done: br-Z1
Done: br-Z2
Done: br-Z3
Done: br-Z4
Done: brk
Done: byn
Done: cad
Done: Cf2-II
Done: Deaf1
Done: Dfd
Done: dl
[KeyError]: dpn is not in PWM_FlyReg
Done: Dref
Done: dsx-F
Done: dsx-M
[KeyError]: dwg is not in PWM_FlyReg
Done: E(spl)
Done: Eip74EF
Done: ems
Done: en
Done: eve
Done: exd
Done: ey
[KeyError]: fkh is not in PWM_FlyReg
Done: ftz
Done: ftz-f1
Done: gl
Done: grh
[KeyError]: gsb is not in PWM_FlyReg
[KeyError]: gsb is not in PWM_FlyReg
[KeyError]: gsb-n is not in PWM_F

In [4]:
# ONLY NEEDS TO DONE ONCE
# Section 3: Write aligned sequences to file and determine ln(p-value) distributions of aligned sequences

# remove patserIn/AlignedSeq directory if it exists
if os.path.exists('matrix'):
    shutil.rmtree('matrix')

## make the matrix files for PATSER
bases = ['A','C','G','T']
for key in PWM:
#    gene = FBidtoName[key]
    if not os.path.exists('matrix'):
        os.makedirs('matrix')
    with open('matrix/%s.txt' %key, 'w') as f:       
        with open('matrix/%s.txt' %key, 'a') as f:
            for i in range(0,4): #for each base
                temp = ['%s |' %bases[i]] + \
                       map(str,np.transpose(PWM[key])[i,:].tolist())
                temp1 = ''.join([x+'\t' for x in temp]) + '\n'
                f.write(temp1)

### Calculate the ln(p-values) of all aligned sequences used to produce the PWMs

# Write aligned sequences associated with PWMs to file
AlignedSeq = {} # dictionary aligned sequences
hasAlignedSeq = [] # list of TFs that have aligned sequences
TF_fullname = []
regex = re.compile('([\w\.\(\)-]+)\s>>_?([\w\.\(\)-]+)') #
regex1 = re.compile('^[ACTG]{2,}$')

with open('Datasets/UmassPGFE_PWMfreq_PublicDatasetD_20160225.txt') as f:
    for line in f:
        m = regex.search(line)
        if m:
            TF_fullname.append(m.group(1)) # full name
            ctr = 1
            if TF_fullname[-1] in filteredPWMnamesR.keys():
                for TF in filteredPWMnamesR[TF_fullname[-1]]:
                    try:
                        os.remove('patserIn/AlignedSeq/%s.txt' %TF)
                    except:
                        pass
        if regex1.search(line):
            if TF_fullname[-1] in filteredPWMnamesR.keys():
                AlignedSeq['%s_%s' %(TF_fullname[-1],ctr)] = line[:-1]
                for TF in filteredPWMnamesR[TF_fullname[-1]]:
                    with open('patserIn/AlignedSeq/%s.txt' %TF, 'a+') as f1:
                            f1.write('%s_%s \ %s \ \n' %(TF_fullname[-1],ctr,line[:-1]))
                    if TF not in hasAlignedSeq:
                        hasAlignedSeq.append(TF)
                    ctr += 1

# Get names of text files in patserIn/AlignedSeq (ONLY NEEDS TO BE DONE ONCE)
_,_,filenames = next(os.walk('patserIn/AlignedSeq'),(None, None, []))
TF_list = [x[:-4] for x in filenames if '.txt' in x]
# Sanity Check: 
# [In]: set(hasAlignedSeq)-set(TF_list)
# [Out]: set()

# Sanity Check:
# [In]: for TF in TF_list:
#             with open('patserIn/AlignedSeq/%s.txt' %TF) as f:
#                 nlines = sum([1 for line in f])
#                 nalignseq = sum([1 for k in AlignedSeq if k.startswith(filteredPWMnames[TF])])
#                 if nlines != nalignseq:
#                     if nalignseq == 0:
#                         if TF == 'dsx':
#                             TF = 'dsx-F'
#                         if nlines != len(od_AlignedSeq_FlyReg[TF]):
#                             print '%s has %d aligned FlyReg sequences but %d sequences in the pasterIn/AlignedSeq file' %(TF,len(od_AlignedSeq_FlyReg[TF]),nlines)
#                     else:
#                         print '%s has %d aligned sequences but %d sequences in the pasterIn/AlignedSeq file' %(TF,nalignseq,nlines)
# [Out]: 

# remove patserOut/AlignedSeq directory if it exists
if os.path.exists('patserOut/AlignedSeq'):
	shutil.rmtree('patserOut/AlignedSeq')
# create/recreate patserOut/AlignedSeq directory
os.makedirs('patserOut/AlignedSeq')

# Run patser using only the TF whose aligned binding sites are in that file
path_to_patser = 'patser-v3e.1/patser-v3e'
for tf in TF_list:
    matrix = 'matrix/%s.txt' %tf # path to TF PWM file
    inputFile = 'patserIn/AlignedSeq/%s.txt' %tf # input sequences
    if not os.path.exists('patserOut/AlignedSeq'):
        os.makedirs('patserOut/AlignedSeq')
    outputFile = 'patserOut/AlignedSeq/AlignedSeq_%s.txt' %tf # PATSER output

    cmd = [path_to_patser, '-A', 'a:t', '0.297', 'c:g', '0.203', \
           '-p', '-m', matrix, '-b', '0', '-d1', '-f', inputFile] 

    process = subprocess.call(cmd,stdout = open(outputFile, 'w'))
    print('Done: %s' %tf)

Done: ab
Done: abd-A
Done: Abd-B
Done: ac
Done: achi
Done: acj6
Done: Adf1
Done: Aef1
Done: al
Done: amos
Done: Antp
Done: aop
Done: ap
Done: ara
Done: Asciz
Done: ase
Done: Atf-2
Done: Atf6
Done: ato
Done: Awh
Done: B-H1
Done: B-H2
Done: bab1
Done: bap
Done: bcd
Done: Bgb
Done: bigmax
Done: bin
Done: Blimp-1
Done: bowl
Done: br
Done: brk
Done: bsh
Done: BtbVII
Done: btd
Done: btn
Done: byn
Done: C15
Done: cad
Done: cato
Done: caup
Done: Cf2
Done: CG10904
Done: CG11085
Done: CG11294
Done: CG11504
Done: CG11617
Done: CG12155
Done: CG12236
Done: CG12605
Done: CG12768
Done: CG15601
Done: CG15696
Done: CG15812
Done: CG18599
Done: CG3065
Done: CG32532
Done: CG33557
Done: CG34031
Done: CG3407
Done: CG3838
Done: CG3919
Done: CG42741
Done: CG4328
Done: CG4360
Done: CG4404
Done: CG4854
Done: CG5180
Done: CG5953
Done: CG6272
Done: CG6276
Done: CG7368
Done: CG7745
Done: CG8281
Done: CG8319
Done: CG8765
Done: CG9876
Done: chinmo
Done: ci
Done: cic
Done: Clk
Done: Coop
Done: crc
Done: CrebA
Done: c

In [5]:
# NEEDS ONLY TO BE RUN ONCE
## Make dictionaries ln(p-value) data and the number of aligned sequences
_,_,filenames = next(os.walk('patserOut/AlignedSeq'),(None, None, []))
lnp_data = {}
nseq = collections.OrderedDict()
regex_tf = re.compile('AlignedSeq_([\w\d\-\(\)\.]+).txt')

if not os.path.exists('patserAnalysis'):
    os.makedirs('patserAnalysis')

# Write lnp distribution data to file in Modified folder
with open('patserAnalysis/lnp_data.txt','w') as f1:
    for file in filenames:
        regex = re.compile('([\w\(\)\.-]+)_(\d+)\s+position=\s+[\dC]+\s+score=\s+[\d\.]+\s+ln\(p-value\)=\s+([\-\.\w]+)')
        # group(1) = gene name
        # group(2) = gene name modifier
        # group(3) = ln(p-value)
        with open('patserOut/AlignedSeq/%s' %file,'r') as f:
            temp = []
            for line in f:
                if regex.search(line):
                    m = regex.search(line)
                    if m:
                        Key = m.group(1)
                        # put ln(p-value) in temporary list
                        temp.append(float(m.group(3)))
            # append temporary list of ln(p-value)s to full list
            if len(temp) > 0:
                m = regex_tf.search(file)
                lnp_data[m.group(1)] = temp
                nseq[m.group(1)] = len(temp)
                f1.write('>%s\n%s\n' %(m.group(1),str(temp)[1:-1]))
            else:
                print file

AlignedSeq_Aef1.txt


In [6]:
# Write PWMs to file
with open('Datasets/Modified/PFMs.fasta','w') as f:
    for TF in lnp_data.keys():
        f.write('>%s\n%s\n' %(TF,"\n".join(" ".join(map(str, [int(item) for item in line])) for line in PWM[TF])))

In [7]:
# Read in the principal TFs involved in early embryogenesis/axis patterning
axis_TFs = []
with open('Datasets/AP_DV_TFs.txt','r') as f:
    for line in f:
        axis_TFs.append(line.split(','))
axis_TFs[0][-1] = axis_TFs[0][-1].strip('[]')[:-1]

for i in [0,1]:
    remove = []
    for tf in axis_TFs[i]:
        if tf not in PWM.keys():
            remove.append(tf)
            print 'No PWM for %s.' %tf
    for item in remove:
        axis_TFs[i].remove(item)

No PWM for croc.
No PWM for Stat92E.
No PWM for tsh.
No PWM for Dad.


In [8]:
# Write AP/DV TFs to file in Modified folder
axis = ['AP','DV']
with open('Datasets/Modified/AP_DV_TFs.txt','w') as f:
    for i in range(2):
        f.write('>%s\n%s\n' %(axis[i],','.join(map(str,axis_TFs[i]))))

In [9]:
# Write TFs by Stage to file in Modified folder
# Determine which TFs are expressed during which stage based on in situ data
Stage = ['1_3','4_6','7_8','9_10','11_12','13_16']
insitu = pd.read_csv('Datasets/insitu_annot.csv',names=['gene_symb','CG','FBgn','stage','staining'])
insitu_ns = insitu[insitu.staining != 'no staining']
TF_stage = {}
FBgn_notfound = []
with open('Datasets/Modified/TFs_by_stage.txt','w') as f:
    for stage in insitu['stage'].unique():
        temp_stage = Stage[stage-1]
        TF_stage[temp_stage] = []
        for FBgn in insitu_ns.FBgn[insitu_ns.stage == stage]:
            try:
                if (type(FBgn)==str) and ('FBgn' in FBgn) and (len(geneID[FBgn]) == 1):
                    if (geneID[FBgn][0] in lnp_data.keys()) and not (geneID[FBgn][0] in TF_stage[temp_stage]):
                        TF_stage[temp_stage].append(geneID[FBgn][0])
            except KeyError:
                if not (FBgn in FBgn_notfound):
                    FBgn_notfound.append(FBgn)
        TF_stage[temp_stage] = sorted(TF_stage[temp_stage],key=lambda t:t.lower())
        f.write('>%s\n%s\n' %(temp_stage,','.join(map(str, TF_stage[temp_stage]))))
[len(TF_stage[stage]) for stage in Stage]

[188, 216, 202, 219, 265, 276]

In [10]:
# Processing the minimal Vienna Tile enhancers
# Read in the minimal VT enhancers
VT_ALL = pd.read_csv('Datasets/2014-01-00083C-Supplementary Table 1.csv')
minVT_ALL = pd.read_csv('Datasets/2014-01-00083C-Supplementary Table 3.csv')
minVT_ALL.head(5)

,VTID,Chromosome,Start,End,Refined element ID,Chromosome.1,Start.1,End.1
0,VT0005,chr2L,15507,17836,VT0005.1,chr2L,15507,15809
1,VT0005,chr2L,15507,17836,VT0005.2,chr2L,16220,17836
2,VT0006,chr2L,16836,18924,VT0006.1,chr2L,16836,18924
3,VT0007,chr2L,18031,20238,VT0007.1,chr2L,18031,19325
4,VT0007,chr2L,18031,20238,VT0007.2,chr2L,19695,19913


In [11]:
# Only keep the VT enhancers that have been minimized, are positive and have been verified
# Note: Positive  = at least one annotation term with strength > 1 
#                   (excluding enhancers with only very weak (1) activities).
# (Identity) Verified = confirmed using PCR of genomic DNA isolated from transgenic flies followed by Sanger sequencing 
#                       (includes 96% of the lines; the remaining 4% were either not tested or failed).
ctr1 = 0
with open('Datasets/Modified/VTminimizedOnly.bed','w') as f:
    for index, row in minVT_ALL.iterrows():
        if not (row.Start == row['Start.1'] and row.End == row['End.1']):
            if (VT_ALL.Verification_status[VT_ALL.VTID == row['VTID']] == 'correct').bool():
                if (VT_ALL.Positive[VT_ALL.VTID == row['VTID']] == 1).bool():
                    ctr1 += 1
                    f.write('%s\t%s\t%s\t%s\n' %(row['Chromosome.1'],row['Start.1'],row['End.1'],row['Refined element ID']))
ctr1

3580

In [12]:
# Sanity check: go from VT to minimized VT
ctr = 0
for index, row in VT_ALL.iterrows():
    if row.Positive == 1 and row.Verification_status == 'correct':
        temp = minVT_ALL[minVT_ALL.VTID == row.VTID]
        if temp.shape[0] > 1:
            ctr += temp.shape[0]
        if temp.shape[0] == 1:
            if (temp.Start == temp['Start.1']).bool(): 
                if (temp.End != temp['End.1']).bool():
                    ctr +=1
            if (temp.End == temp['End.1']).bool():
                if (temp.Start != temp['Start.1']).bool():
                    ctr += 1
            if (temp.Start != temp['Start.1']).bool():
                if (temp.End != temp['End.1']).bool():
                    ctr += 1
ctr

3580

In [13]:
# Liftover from dm3 to dm6
# ./liftOver Datasets/Modified/VTminimizedOnly.bed Datasets/dm3ToDm6.over.chain.gz Datasets/Modified/VTminimizedOnly_dm6.bed Datasets/Modified/VTminimizedOnly_unlifted.bed

# Liftover from dm3 to dm6
command =['./liftOver', \
          'Datasets/Modified/VTminimizedOnly.bed', \
          'Datasets/dm3ToDm6.over.chain.gz', \
          'Datasets/Modified/VTminimizedOnly_dm6.bed', \
          'Datasets/Modified/VTminimizedOnly_unlifted.bed']
process = subprocess.call(command)
# Sanity check: no coordinates unlifted
if os.stat('Datasets/Modified/VTminimizedOnly_unlifted.bed').st_size==0:
    print 'No coordinates unlifted.'
else:
    print 'Some coordinates unlifted!'

# Pick out the sequences from dm6
dm6 = readFASTA('Datasets/dm6.fa','sequences',0,geneID,0,0)
with open('Datasets/Modified/VTminimizedOnly_dm6.fa','w') as f1:
    with open('Datasets/Modified/VTminimizedOnly_dm6.bed','r') as f:
        for line in f:
            temp = line[:-1].split('\t')
            f1.write('>%s\n%s\n' %(temp[3], dm6[temp[0]][int(temp[1])-1:int(temp[2])]))

No coordinates unlifted.


In [14]:
# Sort VT enhancers by stage in which they are expressed
minVT_enhancers_ALL = readFASTA('Datasets/Modified/VTminimizedOnly_dm6.fa','sequences',0,geneID,0)
minVT_enhancers = [] # initialize list of lists of minimized VT enhancers by stage
for stage in ['4_6','7_8','9_10','11_12','13_14','15_16']:
    temp = {k:v for k,v in minVT_enhancers_ALL.iteritems() for vt in VT_ALL.VTID[pd.notnull(VT_ALL['stg%s' %stage])] if vt in k}
    if stage == '15_16': # combine stages 13-14 and 15-16 to match with in situ staging
        minVT_enhancers[-1].update(temp) # merges active enhancers in stages 13-14 and 15-16
    else:
        minVT_enhancers.append(temp)

Stage = ['4_6','7_8','9_10','11_12','13_16']
with open('Datasets/Modified/VTminimizedbyStage.txt', 'w') as f:
    for i in range(len(minVT_enhancers)):
        f.write('>%s\n%s\n' %(Stage[i],','.join(minVT_enhancers[i].keys())))

In [15]:
# Define heterodimers and write to file in Modified folder
# Heterodimers with da
forward = {}
forward[0] = ['ase','Fer3','Hand','nau']
forward[-1] = ['ac','ato','CG33557','Fer2','l(1)sc','sc']
forward[-2] = ['amos','cato','da','Fer1','HLH54F','net','twi','tx']
forward[-3] = ['dimm','Oli']
forward[-4] = ['sage','tap']
forward[-5] = ['HLH4C']

pair1 = ['Mondo','bigmax','Jra','kay','CG6272','Xrp1','crc','Max','Myc','Mnt','tai','Clk','cyc','gce','tgo','dysf','sima','sim','ss']
pair2 = ['bigmax','Mondo','kay','Jra','Xrp1','CG6272','CG6272','Myc','Max','Max','Clk','tai','Clk','Clk','dysf','tgo','tgo','tgo','tgo']
heterodimers = dict(zip(pair1,pair2))
for k in [subitem for item in forward.values() for subitem in item]:
    heterodimers[k] = 'da'

with open('Datasets/Modified/heterodimers.txt','w') as f:
    for k,v in heterodimers.iteritems():
        f.write('>%s\n%s\n' %(k,v))

In [19]:
## Look at overlap between axis patterning and the minimal Vienna Tile enhancers
# Write D. mel AP/DV enhancers to fasta files (input for blat to determine genomic coordinates of enhancers)
AP_enhancers = readFASTA('Datasets/ap_enc_12.fa','sequences',0,geneID,re.compile('([\w\-]+)_mel'))
DV_enhancers = readFASTA('Datasets/dv_enc_12.fa','sequences',0,geneID,re.compile('([\w\-]+)_mel'))

with open('Datasets/Modified/AP_enhancers_mel.fa', 'w') as f:
    for k,v in AP_enhancers.iteritems():
        f.write('>%s\n%s\n' %(k,v))
with open('Datasets/Modified/DV_enhancers_mel.fa', 'w') as f:
    for k,v in DV_enhancers.iteritems():
        f.write('>%s\n%s\n' %(k,v))

In [23]:
# Run AP/DV enhancers through Blat to determine their genomic coordinates

#./blat Datasets/dm6.fa Datasets/Modified/AP_enhancers_mel.fa AP_enhancers_mel_coord.psl

command = []
command.append(['./blat', \
                'Datasets/dm3.2bit', \
                'Datasets/Modified/AP_enhancers_mel.fa', \
                'Datasets/Modified/AP_enhancers_mel_coord.psl'])
command.append(['./blat', \
                'Datasets/dm3.2bit', \
                'Datasets/Modified/DV_enhancers_mel.fa', \
                'Datasets/Modified/DV_enhancers_mel_coord.psl'])
for cmd in command:
    process = subprocess.call(cmd)

In [46]:
regex = re.compile('(\d+)\t\d+\t\d+\t\d+\t\d+\t\d+\t\d+\t\d+\t[\+\-]\t([\w\d\-]+)\t(\d+)\t\d+\t\d+\t(chr[\d\w]+)\t\d+\t(\d+)\t(\d+)')

for axis in ['AP','DV']:
    coordinates = {}
    match_distance = {}
    with open('Datasets/Modified/%s_enhancers_mel_coord.bed' %axis,'w') as f:
        with open('Datasets/Modified/%s_enhancers_mel_coord.psl' %axis,'r') as f1:
            for line in f1:
                m = regex.search(line)
                # group(1) = match length
                # group(2) = enhancer name
                # group(3) = query length
                # group(4) = chromosome
                # group(5) = start
                # group(6) = end
                if m:
                    if m.group(2) not in match_distance.keys() or \
                    m.group(2) in match_distance.keys() and int(m.group(1)) - int(m.group(3)) > match_distance[m.group(2)]:
                            match_distance[m.group(2)] = int(m.group(1)) - int(m.group(3))
                            coordinates[m.group(2)] = '%s\t%s\t%s\t%s\n' %(m.group(4),m.group(5),m.group(6),m.group(2))
        for k,v in coordinates.iteritems():
            f.write(v)
            if match_distance[k] != 0:
                print '%s has a match distance of %d' %(k,match_distance[k])

mes5 has a match distance of -1


In [47]:
# Liftover from dm3 to dm6
for axis in ['AP','DV']:
    command =['./liftOver', \
              'Datasets/Modified/%s_enhancers_mel_coord.bed' %axis, \
              'Datasets/dm3ToDm6.over.chain.gz', \
              'Datasets/Modified/%s_enhancers_mel_coord_dm6.bed' %axis, \
              'Datasets/Modified/%s_enhancers_mel_coord_unlifted.bed' %axis]
    process = subprocess.call(command)
    # Sanity check: no coordinates unlifted
    if os.stat('Datasets/Modified/VTminimizedOnly_unlifted.bed').st_size==0:
        print 'No coordinates unlifted.'
    else:
        print 'Some coordinates unlifted!'

No coordinates unlifted.
No coordinates unlifted.


In [52]:
dv_enh_pat.shape

(39, 4)

In [48]:
## Look at the overlap between the AP/DV and the minimal VT enhancers
# Read in the coordinates for the AP enhancers
ap_enh_pat = pd.read_csv('Datasets/Modified/AP_enhancers_mel_coord_dm6.bed',sep = '\t',header=None)
ap_enh_pat.columns = ['chr','start','end','enhancer']
ap_enh_pat[['start','end']] = ap_enh_pat[['start','end']].astype(int)
ap_enh_pat = ap_enh_pat.sort_values('enhancer') # alphabetize by enhancer name
# Read in the coordinates for the DV enhancers
dv_enh_pat = pd.read_csv('Datasets/Modified/DV_enhancers_mel_coord_dm6.bed',sep = '\t',header=None)
dv_enh_pat.columns = ['chr','start','end','enhancer']
dv_enh_pat[['start','end']] = dv_enh_pat[['start','end']].astype(int)
dv_enh_pat = dv_enh_pat.sort_values('enhancer') # alphabetize by enhancer name
# Read in the coordinates for the minimized VT enhancers
minVT_enh = pd.read_csv('Datasets/Modified/VTminimizedOnly_dm6.bed',sep = '\t',header=None)
minVT_enh.columns = ['chr','start','end','enhancer']
minVT_enh[['start','end']] = minVT_enh[['start','end']].astype(int)
minVT_enh = minVT_enh.sort_values('enhancer') # alphabetize by enhancer name
minVT_enh.head(5)

,chr,start,end,enhancer
0,chr2L,18031,19325,VT0007.1
1,chr2L,19695,19913,VT0007.2
2,chr2L,54362,54503,VT0025.1
3,chr2L,55162,55359,VT0025.2
4,chr2L,243746,243866,VT0108.1


In [49]:
# Table S2: Overlap between axis patterning and the minimal Vienna Tile enhancers
columns = ['Papatsenko', 'Stark', 'Percent_Overlap', 'Stage4_6']
matches = pd.DataFrame(data=np.zeros((0,len(columns))), columns=columns) 
i = 0
for index,row in ap_enh_pat.iterrows():
    for index1,row1 in minVT_enh[minVT_enh.chr == row.chr].iterrows():
        if len(set(range(row.start,row.end)).intersection(range(row1.start,row1.end))) > 0:
            i += 1
            match_len = 100*float(len(set(range(row.start,row.end)).intersection(range(row1.start,row1.end))))/len(range(row.start,row.end))
            match_stage = (row1.enhancer in minVT_enhancers[0])
            matches.loc[i] = [row.enhancer, row1.enhancer, '%.1f' %match_len, '%s' %['yes' if int(match_stage) else 'no'][0]]
for index,row in dv_enh_pat.iterrows():
    for index1,row1 in minVT_enh[minVT_enh.chr == row.chr].iterrows():
        if len(set(range(row.start,row.end)).intersection(range(row1.start,row1.end))) > 0:
            i += 1
            match_len = 100*float(len(set(range(row.start,row.end)).intersection(range(row1.start,row1.end))))/len(range(row.start,row.end))
            match_stage = (row1.enhancer in minVT_enhancers[0])
            matches.loc[i] = [row.enhancer, row1.enhancer, '%.1f' %match_len, '%s' %['yes' if int(match_stage) else 'no'][0]]
matches

,Papatsenko,Stark,Percent_Overlap,Stage4_6
1,ems-up,VT41284.1,15.8,yes
2,eve-15,VT14368.1,70.6,yes
3,eve-37,VT14361.2,78.7,yes
4,eve-37,VT14362.1,100.0,yes
5,eve-46,VT14366.1,74.3,yes
6,eve-46,VT14367.1,93.9,yes
7,eve-M,VT14367.2,13.8,yes
8,eve-M,VT14367.3,39.2,yes
9,eve-M,VT14368.1,55.6,yes
10,ftz-15,VT37571.1,43.2,yes


In [56]:
matches.to_csv('Datasets/Modified/Dataset_Overlap.csv',sep=',')